# Opdracht 3 Soort van YOLO

YOLO
je hebt een grid en bepaald wat is de kans dat dit celletje de centrale cel is van bijvoorbeedl een plaatsje van een hond. Het model doet een voorspelling. YOLO is een pytorch ding. we gebruiken het model dat al gebruikt is om images te detecteren. Kijken waar er bounding boxes zijn. missende data, de totale oppervlak van het plaatjem hoeveel wordt opgevuld door tegels?
Oppervlak keer 250? en dan krijg je de opvulling, check of dit correct is. per plaatje breedte en hoogte bepalen. Per afbeelding een bestand maken voor de labels directory. Coords-idc.txt is een plaatje en de bounding box van tumoren. linksboven naar rechtsonder. 

### Laad libraries

In [1]:
import os
import pandas as pd
import shutil
import random

### Data omzetten naar YOLO-formaat

In [2]:
# Functie om annotaties om te zetten naar YOLO-formaat
def convert_to_yolo_format(coords_file, images_folder, output_folder):
    """
    Converteert annotaties van een CSV-bestand naar YOLO-formaat.

    Parameters:
    - coords_file (str): Pad naar het CSV-bestand met annotaties.
    - images_folder (str): Map met afbeeldingsbestanden.
    - output_folder (str): Map om de YOLO-annotaties op te slaan.

    Returns:
    None
    """
    # Maak output map aan als die nog niet bestaat
    os.makedirs(output_folder, exist_ok=True)

    # Laad annotaties
    coords = pd.read_csv(coords_file, header=None, names=["image_id", "xmin", "ymin", "xmax", "ymax"])

    for image_id in coords["image_id"].unique():
        # Selecteer alle annotaties voor een afbeelding
        image_coords = coords[coords["image_id"] == image_id]
        
        # Bereken de afmetingen van de afbeelding
        image_path = os.path.join(images_folder, f"{image_id}.jpeg")
        if not os.path.exists(image_path):
            print(f"Afbeelding {image_path} niet gevonden.")
            continue
        
        from PIL import Image
        with Image.open(image_path) as img:
            width, height = img.size

        # Genereer YOLO-annotaties
        yolo_annotations = []
        for _, row in image_coords.iterrows():
            x_center = ((row["xmin"] + row["xmax"]) / 2) / width
            y_center = ((row["ymin"] + row["ymax"]) / 2) / height
            bbox_width = (row["xmax"] - row["xmin"]) / width
            bbox_height = (row["ymax"] - row["ymin"]) / height
            yolo_annotations.append(f"1 {x_center} {y_center} {bbox_width} {bbox_height}")

        # Schrijf annotaties naar bestand
        annotation_path = os.path.join(output_folder, f"{image_id}.txt")
        with open(annotation_path, "w") as f:
            f.write("\n".join(yolo_annotations))

# Bestandspaden
coords_file = "../data/coords-idc.txt"
images_folder = "../data/complete_images"
output_folder = "../data/yolo_labels"
convert_to_yolo_format(coords_file, images_folder, output_folder)

### Directory-structuur creëren

In [3]:
def create_dataset_structure(images_folder, labels_folder, output_folder, train_ratio=0.8):
    """
    Creëert een gestructureerde dataset met een training- en validatieset.

    Parameters:
    - images_folder (str): Map met afbeeldingsbestanden.
    - labels_folder (str): Map met YOLO-annotaties.
    - output_folder (str): Doelmap voor gestructureerde dataset.
    - train_ratio (float): Ratio van training-data tot validatie-data.

    Returns:
    None
    """
    # Maak mapstructuur
    for split in ["train", "val"]:
        os.makedirs(os.path.join(output_folder, "images", split), exist_ok=True)
        os.makedirs(os.path.join(output_folder, "labels", split), exist_ok=True)

    # Verdeel data in training- en validatiesets
    image_files = [f for f in os.listdir(images_folder) if f.endswith(".jpeg")]
    random.shuffle(image_files)
    train_split = int(len(image_files) * train_ratio)

    splits = {
        "train": image_files[:train_split],
        "val": image_files[train_split:]
    }

    for split, files in splits.items():
        for image_file in files:
            # Verplaats afbeelding
            src_image = os.path.join(images_folder, image_file)
            dest_image = os.path.join(output_folder, "images", split, image_file)
            shutil.copy(src_image, dest_image)

            # Verplaats corresponderend label
            label_file = image_file.replace(".jpeg", ".txt")
            src_label = os.path.join(labels_folder, label_file)
            dest_label = os.path.join(output_folder, "labels", split, label_file)
            if os.path.exists(src_label):
                shutil.copy(src_label, dest_label)

images_folder = "../data/complete_images"
labels_folder = "../data/yolo_labels"
output_folder = "../data/dataset"
create_dataset_structure(images_folder, labels_folder, output_folder)

### YOLO-configuratiebestand

Maak een configuratie-bestand dataset.yaml met daarin de volgende gegevens

train: /path/to/dataset/images/train

val: /path/to/dataset/images/val

nc: 1  # aantal klassen

names: ['tumor']  # lijst van klasse-namen

### Model trainen

In [5]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
model.train(data="dataset.yaml", epochs=10, single_cls=True, amp=False, verbose=True)

100%|██████████| 6.25M/6.25M [00:00<00:00, 13.5MB/s]


Ultralytics 8.3.51  Python-3.12.7 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train25, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=False, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True,

train: Scanning C:\Users\isabe\Documents\Hanze\jaar_3_new\kw2\modelleren_van_kanker\casus_d\data\dataset\labels\train... 277 images, 0 backgrounds, 1 corrupt: 100%|██████████| 277/277 [00:00<00:00, 727.61it/s]

train: WARNING  C:\Users\isabe\Documents\Hanze\jaar_3_new\kw2\modelleren_van_kanker\casus_d\data\dataset\images\train\12906.jpeg: ignoring corrupt image/label: negative label values [  -0.014904    -0.35914   -0.036478    -0.22465    -0.38828]
train: New cache created: C:\Users\isabe\Documents\Hanze\jaar_3_new\kw2\modelleren_van_kanker\casus_d\data\dataset\labels\train.cache



val: Scanning C:\Users\isabe\Documents\Hanze\jaar_3_new\kw2\modelleren_van_kanker\casus_d\data\dataset\labels\val... 141 images, 1 backgrounds, 0 corrupt: 100%|██████████| 142/142 [00:00<00:00, 358.70it/s]

val: New cache created: C:\Users\isabe\Documents\Hanze\jaar_3_new\kw2\modelleren_van_kanker\casus_d\data\dataset\labels\val.cache


Plotting labels to runs\detect\train25\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train25
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10      4.34G      2.733      4.043      2.572         10        640: 100%|██████████| 18/18 [00:06<00:00,  2.97it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/5 [00:00<?, ?it/s]


NotImplementedError: Could not run 'torchvision::nms' with arguments from the 'CUDA' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'torchvision::nms' is only available for these backends: [CPU, Meta, QuantizedCPU, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, AutogradMeta, Tracer, AutocastCPU, AutocastXPU, AutocastMPS, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PreDispatch, PythonDispatcher].

CPU: registered at C:\actions-runner\_work\vision\vision\pyt

### Voorspellingen doen

In [ ]:
def make_predictions(model_path, test_images_folder, output_folder):
    pass